Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.56 ms, sys: 8.1 ms, total: 12.7 ms
Wall time: 17.1 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7109/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 352 ms, sys: 24.6 ms, total: 376 ms
Wall time: 2.25 s


[1.6676902520661987,
 1.4609210674401938,
 1.1139566633395324,
 1.4056423193979943,
 1.1114363623947858,
 1.8407051105691554,
 1.1399825038649394,
 1.775731806128612,
 1.5961841687408413,
 1.2579610697951122,
 1.976224545815943,
 1.6957730061682263,
 1.8088757424832544,
 1.2620668905647956,
 1.118837493361735,
 1.4900613662406927,
 1.6683307143160637,
 1.0754661183567706,
 1.2558883899702726,
 1.7912222013621704,
 1.3278602441100595,
 1.3781796135730764,
 1.6375348637839304,
 1.0026604530215104,
 1.6862777567414304,
 1.6311705718162783,
 1.8901761124348972,
 1.4237413768416651,
 1.887564657251787,
 1.5028374096432202,
 1.64351817272477,
 1.668564172693018,
 1.0241150326799549,
 1.8420533819924454,
 1.7238963218189944,
 1.9997679623220923,
 1.5976428571907215,
 1.104894128667354,
 1.344481737408274,
 1.258605863508344,
 1.7016865009678872,
 1.5764400878720046,
 1.7648628657166343,
 1.5413833029778599,
 1.612527255582577,
 1.3050775205188612,
 1.396034129877693,
 1.5659795587930976,
 1.4

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.20907984331816,
 1.4878290528557625,
 1.6879795726128897,
 1.5825401416875884,
 1.7728426327388631,
 1.424570440046845,
 1.7051369655864592,
 1.9392486548009085,
 1.0486402446916152,
 1.4042996527170157,
 1.631039243895485,
 1.542482639041597,
 1.899089380317886,
 1.6885773278277452,
 1.5791821117277907,
 1.6370418529291422,
 1.600988803378097,
 1.8246719848171364,
 1.8180957092672732,
 1.44532000014354,
 1.3254851976837587,
 1.1697331814349423,
 1.157180976347961,
 1.3893169164345172,
 1.5239089525778482,
 1.4702953195854371,
 1.340550682372652,
 1.7859787117723154,
 1.7423684946096252,
 1.3108195851047562,
 1.3924674437706028,
 1.1832816005994895,
 1.2150790926819486,
 1.7884058896699186,
 1.3696953438409714,
 1.190713442085972,
 1.6387619385465326,
 1.195978954903309,
 1.5491058296974334,
 1.7181581488097404,
 1.571302538632879,
 1.9326505978217323,
 1.7439897866120027,
 1.4026141358460449,
 1.1809840131623668,
 1.8283453356029846,
 1.6120232483897308,
 1.8861227186590985,
 1.573

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 281 ms, sys: 42 ms, total: 323 ms
Wall time: 2.29 s
